# **Simple RAG app with conversation History**

## *Initial code*

In [2]:

import os
from dotenv import load_dotenv


load_dotenv()

gemini_api_key = os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [3]:

from langchain_google_genai import ChatGoogleGenerativeAI


llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=gemini_api_key)


In [4]:

from langchain_core.messages import HumanMessage


llm.invoke([HumanMessage(content="What is the capital of Andhra Pradesh?")])

AIMessage(content="The designated capital of Andhra Pradesh is **Amaravati**.\n\nHowever, it's important to note the history and recent developments:\n\n*   **Hyderabad** served as the joint capital for both Andhra Pradesh and Telangana until June 2, 2024, as per the Andhra Pradesh Reorganisation Act, 2014.\n*   Andhra Pradesh had plans to develop Amaravati as its new capital city.\n*   There have been proposals and legal discussions about having three capitals (Visakhapatnam for executive, Amaravati for legislative, and Kurnool for judicial), but as of now, **Amaravati** remains the official designated capital.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--0cd03544-03ee-4d0d-85a4-c276ad49ee29-0', usage_metadata={'input_tokens': 9, 'output_tokens': 771, 'total_tokens': 780, 'input_token_details': {'cache_read': 0}, 'output_token_details':

## Chat-History demo

In [5]:

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_history = RunnableWithMessageHistory(llm, get_session_history)


### *first - configuration*

In [6]:

config = { "configurable": { "session_id": "demo1" } }


In [7]:

# Providing Information
response = with_history.invoke(
    [HumanMessage(content="Hi! I am Sri, final year student at CVR COllege of Engineering.")],
    config=config
)

response.content

"Hi Sri! Nice to meet you. It's great to connect with a final year student from CVR College of Engineering.\n\nHow can I help you today? What's on your mind?"

In [8]:

# Asking a Question
with_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config
).content


'Your name is **Sri**! You mentioned it in your first message.\n\nHow can I help you, Sri?'

### *second - configuration*

In [9]:

config1 = { "configurable": { "session_id": "demo2" } }

In [10]:

# Asking a Question with a different session
with_history.invoke(
    [HumanMessage(content="what is my name?")],
    config=config1
).content


"As an AI, I don't have access to any personal information about you, so I don't know your name.\n\nYou haven't told me!"

In [ ]:

# Providing Information to a demo2 session
with_history.invoke(
    [HumanMessage(content="My name is Riyaz.")],
    config=config1
).content

"Okay, Riyaz. It's nice to meet you! How can I help you today?"

In [ ]:

# Asking a Question
with_history.invoke(
    [HumanMessage(content="what is my name?")],
    config=config1
).content


'Your name is Riyaz.'

## ChatPrompt Template

In [56]:

# Adding ChatPromptTemplate and bit complexity...
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [    ("system", "You are a friendly assistant. Answer all questions in {language}."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | llm

with_history = RunnableWithMessageHistory( chain, get_session_history, input_messages_key="messages" )


In [57]:

config2 = { "configurable": { "session_id": "demo2" } }

with_history.invoke(
    {"messages": [HumanMessage(content="Hi! I am Avinash.")], "language": "Telugu"},
    config=config2
).content


'హాయ్ అవినాష్! మిమ్మల్ని కలవడం నాకు చాలా సంతోషంగా ఉంది. నేను మీకు ఎలా సహాయపడగలను?'

In [58]:

with_history.invoke(
    {"messages": [HumanMessage(content="what is my name?")], "language": "Hindi"},
    config=config2
).content


'आपका नाम अविनाश है।'

## Trimming Messages

In [65]:

from langchain_core.messages import SystemMessage, trim_messages, AIMessage

trimmer = trim_messages(
    max_tokens=40,
    strategy="last",
    token_counter = llm,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages = [
    SystemMessage(content="You are a friend."),
    HumanMessage(content="Hi! I am Rohith."),
    AIMessage(content="Hello!"),
    HumanMessage(content="I like to play Badminton."),
    AIMessage(content="Nice!"),
    HumanMessage(content="What is capital of Telangana?"),
    AIMessage(content="Hyderabad"),
    HumanMessage(content="Thanks!"),
    AIMessage(content="You are welcome!"),
    HumanMessage(content="Having fun?"),
    AIMessage(content="Yes!")
]

trimmer.invoke(messages)


[SystemMessage(content='You are a friend.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is capital of Telangana?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hyderabad', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Thanks!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='You are welcome!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Yes!', additional_kwargs={}, response_metadata={})]

In [ ]:

# As max tokens are 40, the messages are trimmed to fit within the limit. Therefore It doesn't remember my name.
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer) | prompt | llm    
)

chain.invoke(
    {"messages": messages + [HumanMessage(content="What is my name?")], "language": "English" }
)


AIMessage(content="As a large language model, I have no memory of past conversations. Therefore, I do not know your name. You can tell me if you'd like! 😊", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--b9d706e6-1ebf-49f4-a012-370b1eff5d09-0', usage_metadata={'input_tokens': 33, 'output_tokens': 35, 'total_tokens': 68, 'input_token_details': {'cache_read': 0}})

In [71]:

chain.invoke(
    {"messages": messages + [HumanMessage(content="What question did I ask?")], "language": "English" }
).content


'You asked "Having fun?".'

In [ ]:

chain.invoke(
    {"messages": messages + [HumanMessage(content="capital of Telangana")], "language": "English" }
).content


'The capital of Telangana is Hyderabad!'

In [75]:

# Wrapping into message history
with_history = RunnableWithMessageHistory(
    chain, get_session_history, input_messages_key="messages"
)

config3 = { "configurable": { "session_id": "demo3" } }

with_history.invoke(
    {"messages": messages + [HumanMessage(content="What question did I ask?")], "language": "English" },
    config=config3
).content


'You asked, "Having fun?"'